배우 평판 정보 테이블을 학습시킬 피처로 전환하는 과정

In [13]:
# 패키지 설치 및 불러오기
pip install googletrans==4.0.0rc1
import pandas as pd
from googletrans import Translator

# 파일 불러오기
reputation_actor_drama = pd.read_csv("reputation_actor_drama.csv")
reputation_actor_movie = pd.read_csv("reputation_actor_movie.csv")

# 각 테이블의 date 컬럼을 datetime 형식으로 변환
reputation_actor_drama['date'] = pd.to_datetime(reputation_actor_drama['date'])
reputation_actor_movie['date'] = pd.to_datetime(reputation_actor_movie['date'])

# 각 배우의 집계된 횟수(count)와 평균 평판지수(avg) 계산
actor_counts_drama = reputation_actor_drama.groupby('actor_name').size()
actor_counts_movie = reputation_actor_movie.groupby('actor_name').size()

reputation_mean_drama = reputation_actor_drama.groupby('actor_name')['reputation_rate'].mean()
reputation_mean_movie = reputation_actor_movie.groupby('actor_name')['reputation_rate'].mean()

# 두 테이블 합치기
reputation_actor = pd.concat([actor_counts_drama, actor_counts_movie, reputation_mean_drama, reputation_mean_movie], axis=1)
reputation_actor.columns = ['drama_count', 'movie_count', 'drama_avg_reputation', 'movie_avg_reputation']

# count 값이 없는 경우 0으로 채우고, count 값이 중복된 경우 합산하기
reputation_actor.fillna(0, inplace=True)
reputation_actor['count'] = reputation_actor['drama_count'] + reputation_actor['movie_count']

# 중복된 배우에 대해 count는 합산, 평판지수는 평균을 계산하여 출력하고 컬럼 정리
reputation_actor['avg_reputation'] = (reputation_actor['drama_avg_reputation'] + reputation_actor['movie_avg_reputation']) / 2
reputation_actor = reputation_actor.drop(['drama_count', 'movie_count', 'drama_avg_reputation', 'movie_avg_reputation'], axis=1)
reputation_actor.reset_index(inplace=True)

# Translator 객체 생성
translator = Translator()

# 한국어 이름을 영어로 번역 (drama테이블의 배우 정보와 일치시키기 위함)
translated_names = []
for name in reputation_actor['actor_name']:
    try:
        translated_name = translator.translate(name, src='ko', dest='en').text
    except Exception as e:
        print(f"Translation error occurred for name '{name}': {e}")
        translated_name = name  # 번역 오류 발생 시 원래 이름을 그대로 사용
    translated_names.append(translated_name)

# 'actor_name_en' 열을 'actor_name'으로 변경
reputation_actor['actor_name'] = translated_names

# 테이블로 저장
reputation_actor.to_csv("reputation_actor.csv", index=False)

SyntaxError: invalid decimal literal (<ipython-input-13-78250529cd62>, line 2)

드라마 테이블 머신러닝 모델 만들기

In [14]:
''' 패키지 불러오기 '''
import pandas as pd
import numpy as np
import json
from pandas import json_normalize
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import RobustScaler
import numpy as np


''' 파일 불러오기 '''
# 드라마 테이블 불러오기
with open('2023-2024-dramadata.json') as f:
  js = json.loads(f.read())
drama = pd.DataFrame(js)

# 배우 평판 테이블 불러오기
reputation_actor = pd.read_csv("reputation_actor.csv")



''' 전처리 (1) 컬럼 필터링 '''
# json형태인 값 풀어주기
drama['genre_names'] = drama['genres'].apply(lambda x: [genre['name'] for genre in x])
drama['network_names'] = drama['networks'].apply(lambda x: [network['name'] for network in x])
drama['production_companie_names'] = drama['production_companies'].apply(lambda x: [production_companie['name'] for production_companie in x])
drama['credit_names'] = drama['credits'].apply(lambda x: [credit['name'] for credit in x['cast']] if 'cast' in x else [])

# 리스트로 된 값 풀어주기
drama['episode_run_time'] = drama['episode_run_time'].apply(lambda x: x[0] if x else None)

# 'first_air_date' 컬럼을 datetime 형식으로 변환
drama['first_air_date'] = pd.to_datetime(drama['first_air_date'])

# 필요없는 컬럼 모두 제거
drama = drama.drop(['backdrop_path', 'homepage', 'id', 'in_production', 'poster_path', 'genres', 'seasons', 'tagline',
                    'created_by', 'last_episode_to_air', 'next_episode_to_air', 'networks', 'spoken_languages',
                    'origin_country', 'original_language', 'production_companies', 'credits', 'production_countries',
                    'overview', 'languages', 'original_name', 'created_by', 'status'], axis=1)

# 컬럼명 변경
drama = drama.rename(columns={'genre_names': 'genres',
                              'network_names': 'networks',
                              'production_companie_names': 'production_companies',
                              'credit_names': 'credits'})



''' 전처리 (2) 결측값 제거 '''
# vote_average와 vote_count 컬럼에서 0 또는 0.000 값을 NaN으로 변환
drama['vote_average'].replace(0.000, float('nan'), inplace=True)
drama['vote_count'].replace(0, float('nan'), inplace=True)

# SimpleImputer를 사용하여 NaN 값을 상수값으로 대체하는 객체 생성
constant_imputer = SimpleImputer(strategy='constant', fill_value=1)

# 컬럼의 NaN 값을 상수값으로 대체
drama['vote_average'] = constant_imputer.fit_transform(drama[['vote_average']])
drama['vote_count'] = constant_imputer.fit_transform(drama[['vote_count']])
drama['episode_run_time'] = constant_imputer.fit_transform(drama[['episode_run_time']])



''' 인코딩 (1) credits 컬럼 인코딩  '''
# 표준화 함수 정의
def standardize_name(name):
    standardized_name = name.replace(" ", "").lower()
    standardized_name = standardized_name.replace("-", "").replace(".", "").replace(",", "")
    return standardized_name

# actor_name 컬럼을 정규화하여 조회
reputation_actor["actor_name_normalized"] = reputation_actor["actor_name"].apply(standardize_name)
drama.to_csv("인코딩 전 데이터.csv", index=False) ##########################################################################################
# credits 컬럼의 값을 avg_reputation 값으로 바꾸기
if "actor_name" in reputation_actor.columns:
    if "credits" in drama.columns:
        # 최솟값 계산
        min_avg_reputation = reputation_actor["avg_reputation"].min()

        # credits 컬럼의 값이 있는지 확인하고 표준화한 후 배우 이름과 일치하는 값 찾기
        for i, row in drama.iterrows():
            standardized_credits = [standardize_name(name) for name in row["credits"]]
            updated_credits = []
            for name in standardized_credits:
                if name in reputation_actor["actor_name_normalized"].values:
                    actor_index = reputation_actor[reputation_actor["actor_name_normalized"] == name].index[0]
                    reputation = reputation_actor.at[actor_index, "avg_reputation"]
                    updated_credits.append(reputation)
                else:
                    # 배우의 값이 없을 경우 최솟값의 avg_reputation 사용
                    updated_credits.append(min_avg_reputation)
            drama.at[i, "credits"] = updated_credits
# credits컬럼을 각 배우들의 평판지수의 합으로 대체
drama['credits'] = drama['credits'].apply(sum)



''' 인코딩 (2) networks,genres, type, adult 컬럼 인코딩  '''
# networks컬럼을 방영되는 채널 개수로 대체
drama['networks'] = drama['networks'].apply(len)

# genres 컬럼 원핫 인코딩
all_genres = set(genre for sublist in drama['genres'] for genre in sublist)
for genre in all_genres:
    drama[genre] = drama['genres'].apply(lambda x: 1 if genre in x else 0)
drama.drop(columns=['genres'], inplace=True)

# get_dummies 함수를 사용하여 type 컬럼 원핫 인코딩
encoded_type = pd.get_dummies(drama['type'], prefix='type')
drama = pd.concat([drama.drop(columns=['type']), encoded_type], axis=1)

# get_dummies 함수를 사용하여 adult 컬럼 원핫 인코딩
encoded_adult = pd.get_dummies(drama['adult'], prefix='adult')
drama = pd.concat([drama.drop(columns=['adult']), encoded_adult], axis=1)



''' 테이블 재졍렬 '''
# 컬럼 순서 재정렬
desired_order = ['name', 'production_companies', 'first_air_date', 'last_air_date'] + \
                [col for col in drama.columns if col not in ['name', 'production_companies', 'first_air_date', 'last_air_date']]
drama = drama.reindex(columns=desired_order)

# first_air_date 열을 기준으로 오름차순으로 정렬하고 인덱스 초기화
drama = drama.sort_values(by='first_air_date').reset_index(drop=True)



'''ML : pularity, vote_average, vote_count 컬럼 예측해보기'''
# 예측할 컬럼 선택
target_column = 'popularity'
# 특징 변수 선택 (다른 컬럼들을 이용하여 예측)
features = ['episode_run_time', 'number_of_episodes', 'number_of_seasons',
            'popularity', 'vote_average', 'vote_count', 'networks', 'credits',
            'Sci-Fi & Fantasy', '범죄', '드라마', '미스터리', 'War & Politics', '코미디', '가족',
            'Action & Adventure', 'type_Miniseries', 'type_Scripted','adult_False']

# Train 및 Test 데이터 준비
train = drama[drama['first_air_date'].dt.year <= 2023]
test = drama[drama['first_air_date'].dt.year >= 2024]

train_etc = train[['name', 'production_companies', 'first_air_date', 'last_air_date']]
train = train.drop(['name', 'production_companies', 'first_air_date', 'last_air_date'], axis=1)

test_etc = test[['name', 'production_companies', 'first_air_date', 'last_air_date']]
test = test.drop(['name', 'production_companies', 'first_air_date', 'last_air_date'], axis=1)

test = test.drop(['popularity', 'vote_average', 'vote_count'], axis=1)
# print(train.shape, test.shape) --> (43, 19) (13, 16)
target = train[['popularity', 'vote_average', 'vote_count']]
train = train.drop(['popularity', 'vote_average', 'vote_count'], axis=1)
# print(train.shape, test.shape) --> (43, 16) (13, 16)

# 특정 컬럼 스케일링 (값이 크고 편차가 많은 컬럼)
cols = ['episode_run_time',	'number_of_episodes', 'credits']
scaler = RobustScaler()
train[cols] = scaler.fit_transform(train[cols])
test[cols] = scaler.transform(test[cols])

# 모델 검증
x_tr, x_val, y_tr, y_val = train_test_split(train,target, test_size=0.15, random_state=200)
# print(x_tr.shape, x_val.shape, y_tr.shape, y_val.shape) -> (36, 16) (7, 16) (36, 3) (7, 3)

# 선형 회귀 모델 사용
model = LinearRegression()
model.fit(x_tr, y_tr)
pred = model.predict(x_val)
mae = mean_absolute_error(np.exp(y_val), np.exp(pred))
print(f'Mean Absolute Error: {mae}')

# 'test_data'에 대한 예측값 추가
pred = model.predict(test)
pred_df = pd.DataFrame(pred, columns=['popularity', 'vote_average', 'vote_count'])
drama = pd.concat([test_etc.reset_index(drop=True), pred_df.reset_index(drop=True)], axis=1)



''' 저장을 위해 테이블 필터링 '''
# production_companies 컬럼을 리스트의 각 요소를 행으로 풀어내기
drama = drama.explode('production_companies')

# 남길 회사명 영어-한글 매핑 딕셔너리
selected_companies = {
    'Studio Dragon': '스튜디오드래곤',
    'CJ ENM': 'CJ ENM',
    'Barunson Studio': '바른손',
    'CJ ENM Studios': 'CJ ENM',
    'Raemongraein': '래몽래인',
    'Samhwa Networks': '삼화네트웍스',
    'Studio S': 'SBS'
}
# 특정 회사만 남기기
drama = drama[drama['production_companies'].isin(selected_companies.keys())]

# 회사명 영어에서 한글로 변경
drama['production_companies'] = drama['production_companies'].map(selected_companies)

# 컬럼 순서 재정렬
desired_order = ['production_companies'] + \
                [col for col in drama.columns if col not in ['production_companies']]
drama = drama.reindex(columns=desired_order)



''' 결과물 얻기 '''
# 각 컬럼의 평균 계산
mean_values = drama[['popularity', 'vote_average', 'vote_count']].mean()

# 긍정/중립/부정을 분류하는 함수 정의
def classify_sentiment(row):
    # 각 컬럼의 값과 평균 비교하여 분류
    if row['popularity'] > mean_values['popularity'] and row['vote_average'] > mean_values['vote_average'] and row['vote_count'] > mean_values['vote_count']:
        return '긍정'
    elif row['popularity'] < mean_values['popularity'] and row['vote_average'] < mean_values['vote_average'] and row['vote_count'] < mean_values['vote_count']:
        return '부정'
    else:
        return '중립'

# 새로운 컬럼 추가
drama['sentiment'] = drama.apply(classify_sentiment, axis=1)




''' EDA '''
# print(drama.columns)
# print(drama['credits'].iloc[2])
# drama['status'].value_counts()
# drama[drama['first_air_date'].dt.year >= 2024]
# drama.isnull().sum()
# drama.to_csv('drama.csv', index=False)


# 결과 출력
drama.to_csv('result_drama.csv', index=False)

Mean Absolute Error: 1.5853052268827368e+114
